In [ ]:
import pandas as pd
import folium

Download data acceslibre

In [ ]:
url_access_libre = "https://www.data.gouv.fr/fr/datasets/r/5b0f44f2-e6ea-4a58-874d-6fe364b40342"
df_accesslibre = pd.read_csv(url_access_libre)

In [ ]:
df_accesslibre.columns

Columns selection

In [ ]:
df_balises = df_accesslibre[['id', 'name', 'code_insee', 'longitude', 'latitude', 'entree_balise_sonore']]


Suppression des établissements sans information sur les balises sonores

In [ ]:
df_balises = df_balises[df_balises['entree_balise_sonore'].notna()]
print(df_balises.shape)

In [ ]:
df_balises['entree_balise_sonore'].value_counts()

In [ ]:
df_balises

Cleaning des positions GPS

In [ ]:
# Convertir les colonnes latitude et longitude si nécessaire
df_balises['longitude'] = pd.to_numeric(df_balises['longitude'], errors='coerce')
df_balises['latitude'] = pd.to_numeric(df_balises['latitude'], errors='coerce')

# Supprimer les lignes avec des coordonnées invalides
df_balises = df_balises.dropna(subset=['longitude', 'latitude'])


On ne garde que les établissements d'une communes, et uniquement ceux avec une borne sonore. 

In [ ]:
df_balises_true = df_balises[df_balises["entree_balise_sonore"]]
df_balises_true = df_balises[df_balises["code_insee"]=="85294"]


In [ ]:

# Définir un point central pour centrer la carte (par exemple, le premier point)
point_central = [df_balises_true['latitude'].mean(), df_balises_true['longitude'].mean()]

# Ajouter des points au marqueur
carte = folium.Map(location=point_central, zoom_start=6)

# Ajouter des points de données comme marqueurs
for _, row in df_balises_true.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['name']}</b>",
    ).add_to(carte)

# Afficher la carte dans le notebook
display(carte)
